# Generating simplified geometries for administrative boundaries

This Jupyter notebook can be used to generate simplified geometries of administrative areas.

**Click on the cell below and click on "Run" to import the necessary libraries.**

In [1]:
from functools import partial
import math
from descartes.patch import PolygonPatch
import pyproj
from pyproj import CRS
from shapely.geometry.geo import box, mapping
from shapely.ops import cascaded_union, transform
import json
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
from ipywidgets import interact, interactive, FloatSlider

%matplotlib inline

BLUE = '#6699cc'
GRAY = '#999999'

proj_epsg4326 = CRS("EPSG:4326")
proj_epsg3857 = CRS("EPSG:3857")

project_fwd = partial(
    pyproj.transform,
    proj_epsg4326,
    proj_epsg3857)

project_bwd = partial(
    pyproj.transform,
    proj_epsg3857,
    proj_epsg4326)

### Part 1: Selection of administrative areas

The input field below the next cell expects Nominatim queries as input. The query can contain multiple places, separated by a semicolon. If no input field is shown, click on the next cell and click on "Run".
You can test your query here: [https://nominatim.openstreetmap.org](https://nominatim.openstreetmap.org).

If the results match your expectations, you can scroll down to Part 2:

Example queries:

* `Solothurn, Switzerland`
* `Kanton Bern, Switzerland`
* `Kanton Bern, Switzerland; Solothurn, Switzerland`

In [3]:
def get_boundaries(query):
    print("Retrieved query: {}".format(query))

    queries = query.split(";")
    results = ox.gdf_from_places(queries)

    geoms = []
    for index, row in results.iterrows():    
        geoms.append(row['geometry'])
    geom = cascaded_union(geoms)

    # Plot query result
    fig, ax = plt.subplots()
    patch_original = PolygonPatch(geom, fc=GRAY, ec=GRAY, alpha=0.5, zorder=1)
    ax.add_patch(patch_original)
    for g in geoms:
        patch_geom = PolygonPatch(g, fc="None", ec='k', alpha=1.0, zorder=2)
        ax.add_patch(patch_geom)
        
    minx, miny, maxx, maxy = geom.bounds
    plt.xlim(minx, maxx)
    plt.ylim(miny, maxy)
    plt.show()
    return geom
 
w = interactive(get_boundaries, {'manual': True},
                query='Solothurn, Switzerland')
display(w)

interactive(children=(Text(value='Solothurn, Switzerland', description='query'), Button(description='Run Inter…

### Part 2: Selecting parameters for simplification

Using the sliders below the next cell, you can adjust the parameters of `box_size` and `buffer_distance`. If no sliders are shown, or the wrong area is displayed, select the next cell and click on "Run".  A simplified geometry is generated by creating rectangular boxes. Only the boxes that touch the original geometry with an extra buffer distance are kept. This method is not particularly efficent. Especially for larger geometries, a slow processing time should be expected.

* `buffer_distance` sets the size of a buffer around an administrative boundary. [Units in  (WGS 84 / Pseudo-Mercator projection)]
* `box_size` sets the size of the "boxes" that are used to generate the simplified geometries. [degree longitude/latitude]

Following the plot of the original, respectively simplified geometry, the simplified geometry is printed in text form: If you are satisfied with results, you can copy this text. 
```
{'coordinates': (((7.48, 47.1),
                  (7.48, 47.08),
                  ...
                  (7.48, 47.1)),),
 'type': 'Polygon'}



In [4]:
geom = w.result

bounds = geom.bounds
max_diff = round(max(bounds[2] - bounds[0], bounds[3] - bounds[1]), 2)
default_box = max(round(max_diff * 0.05, 2), 0.01)
max_box = max(0.1, default_box * 5)

def create_simple_geometry(box_size, buffer_distance):

    print("Processing")
    # Project to EPSG:3857 to apply buffer
    if buffer_distance > 0:
        geom_epsg3857 = transform(project_fwd, geom)
        geom_epsg3857_buffered = geom_epsg3857.buffer(buffer_distance)
        geom_buffered = transform(project_bwd, geom_epsg3857_buffered)
    else:
        geom_buffered = geom

    # Create boxes
    bounds = geom_buffered.bounds
    # To save space, round bounding box to generate coordinates with less digits
    minx = math.floor(bounds[0] / box_size) * box_size
    miny = math.floor(bounds[1] / box_size) * box_size
    maxx = math.ceil(bounds[2] / box_size) * box_size
    maxy = math.ceil(bounds[3] / box_size) * box_size

    boxes = []
    for x in np.arange(minx, maxx, box_size):
        for y in np.arange(miny, maxy, box_size):

            pts = [x, y, x + box_size, y + box_size]
            pts = map(lambda x: round(x, 6), pts)
            b = box(*pts)
            if b.intersects(geom_buffered):
                boxes.append(b)

    new_geom = cascaded_union(boxes)

    # Remove unnecessary points
    new_geom = new_geom.simplify(0.0001, preserve_topology=False)

    geojson = mapping(new_geom)

    # Plot result
    patch_original = PolygonPatch(geom, fc=GRAY, ec=GRAY, alpha=0.3, zorder=1)
    patch_buffer = PolygonPatch(geom_buffered, fc=GRAY, ec=GRAY, alpha=0.3, zorder=1)
    patch_box = PolygonPatch(new_geom, fc=BLUE, ec=BLUE, alpha=0.5, zorder=2)

    fig, ax = plt.subplots()
    ax.add_patch(patch_original)
    ax.add_patch(patch_buffer)
    ax.add_patch(patch_box)

    plt.xlim(minx, maxx)
    plt.ylim(miny, maxy)
    plt.show()

    print(json.dumps(geojson, indent=4))

interactive(create_simple_geometry,
         box_size=FloatSlider(value=default_box, min=0.01, max=max_box, step=0.01, continuous_update=False),
         buffer_distance=FloatSlider(value=500.0, min=0.0, max=5000.0, step=100.0, continuous_update=False))

interactive(children=(FloatSlider(value=0.03, continuous_update=False, description='box_size', max=0.15, min=0…